In [11]:
import sys
import keras
import cv2
import numpy
import matplotlib
import skimage
import os

print(('python: {}'.format(sys.version)))
print(('keras: {}'.format(keras.__version__)))
print(('OpenCV: {}'.format(cv2.__version__)))
print(('Numpy: {}'.format(numpy.__version__)))
print(('Matplotlib: {}'.format(matplotlib.__version__)))
print(('Skimage: {}'.format(skimage.__version__)))


Using TensorFlow backend.


python: 2.7.15rc1 (default, Nov 12 2018, 14:31:15) 
[GCC 7.3.0]
keras: 2.2.4
OpenCV: 3.4.3
Numpy: 1.16.2
Matplotlib: 2.2.4
Skimage: 0.14.2


In [0]:
#import necessary packages
from keras.models  import Sequential
from keras.layers import Conv2D, Input
from keras.optimizers import SGD, Adam
from skimage.measure import compare_ssim as ssim
from matplotlib import pyplot as plt
import cv2
import numpy as np
import math
import os

%matplotlib inline

In [13]:
!ls


3051crop_weight_200.h5	images	images2  output  sample_data  source


In [0]:
#function for PSNR
def psnr(target, ref):
    
    #RGB image
    target_data = target.astype(float)
    ref_data = ref.astype(float)
    
    diff = ref_data - target_data
    diff = diff.flatten('C')
    
    rmse = math.sqrt(np.mean(diff ** 2.))
    
    return 20 * math.log10(255. / rmse)

#function for MSE
def mse(target,ref):
    
    #MSE is the sum of squared diff between the two images
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    err /= float(target.shape[0] * target.shape[1])
    
    return err

#function that combines all three image quaity metrices
def compare_images(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel=True))
    
    return scores


In [0]:
#function for peeparing downgraded images by introducing quality distortion via resizing
def perpare_images(path, factor):
    
    #loop through files
    for file in os.listdir(path):
        
        img = cv2.imread(path + '/' +file)
        
        #find old and new image dimensions
        h, w, c = img.shape
        new_height = h / factor
        new_width = w / factor
        
        #resizing
        #down
        img = cv2.resize(img, (new_width, new_height), interpolation = cv2.INTER_LINEAR)
        
        #up
        img = cv2.resize(img, (w, h), interpolation = cv2.INTER_LINEAR)
        
        
        #save image
        
        print(('Saving {}',format(file)))
        cv2.imwrite('images/{}'.format(file), img)

In [0]:
!mkdir source
!mkdir images
!mkdir images2
!mkdir output

mkdir: cannot create directory ‘source’: File exists
mkdir: cannot create directory ‘images’: File exists
mkdir: cannot create directory ‘images2’: File exists
mkdir: cannot create directory ‘output’: File exists


In [16]:
perpare_images('source/', 2)

('Saving {}', 'ppt3.bmp')
('Saving {}', 'foreman.bmp')
('Saving {}', 'coastguard.bmp')
('Saving {}', 'baboon.bmp')
('Saving {}', 'face.bmp')
('Saving {}', 'flowers.bmp')
('Saving {}', 'barbara.bmp')
('Saving {}', 'lenna.bmp')
('Saving {}', 'muskan.jpeg')
('Saving {}', 'comic.bmp')
('Saving {}', 'monarch.bmp')
('Saving {}', 'pepper.bmp')
('Saving {}', 'zebra.bmp')


In [0]:
#testing the generated images using image quality metrices

for file in os.listdir('images/'):
    
    target = cv2.imread('images/{}'.format(file))
    ref = cv2.imread('source/{}'.format(file))
    
    scores = compare_images(target, ref)
    
    print(('{}\nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(file, scores[0], scores[1], scores[2])))

ppt3.bmp
PSNR: 24.8492616895
MSE: 638.668426391
SSIM: 0.928402394232

foreman.bmp
PSNR: 30.1445653266
MSE: 188.688348327
SSIM: 0.933268417389

coastguard.bmp
PSNR: 27.1616006639
MSE: 375.008877841
SSIM: 0.756950063355

baboon.bmp
PSNR: 22.1570840834
MSE: 1187.11613333
SSIM: 0.6292775879

face.bmp
PSNR: 30.9922065029
MSE: 155.231897185
SSIM: 0.800843949229

flowers.bmp
PSNR: 27.4545048054
MSE: 350.550939227
SSIM: 0.869728628697

barbara.bmp
PSNR: 25.9066298376
MSE: 500.655085359
SSIM: 0.809863264641

lenna.bmp
PSNR: 31.4734929787
MSE: 138.948005676
SSIM: 0.846098920052

muskan.jpeg
PSNR: 39.2802820054
MSE: 23.0236097332
SSIM: 0.943347084847

comic.bmp
PSNR: 23.7998615022
MSE: 813.233883657
SSIM: 0.83473354164

monarch.bmp
PSNR: 30.1962423653
MSE: 186.456436157
SSIM: 0.943957429343

pepper.bmp
PSNR: 29.8894716169
MSE: 200.103393555
SSIM: 0.835793756846

zebra.bmp
PSNR: 27.9098406393
MSE: 315.658545953
SSIM: 0.891165620933



In [0]:
#define the SRCNN model

def model():
    
    #define model type
    SRCNN = Sequential()
    
    #add model layers
    SRCNN.add(Conv2D(filters=128, kernel_size=(9,9),kernel_initializer='glorot_uniform', activation='relu', padding='valid', use_bias=True, input_shape=(None, None, 1)))
    
    SRCNN.add(Conv2D(filters=64, kernel_size = (3,3), kernel_initializer='glorot_uniform', activation='relu', padding='valid', use_bias=True))
    
    SRCNN.add(Conv2D(filters=1, kernel_size = (5,5), kernel_initializer='glorot_uniform', activation='relu', padding='valid', use_bias=True))
    
    #define optimizer
    adam = Adam(lr=0.0003)
    
    #compile model
    SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])
    
    return SRCNN

In [0]:
#define necessary image processing functions

def modcrop(img, scale):
    
    tmpsz = img.shape
    sz = tmpsz[0:2]
    sz = sz - np.mod(sz, scale)
    img = img[0:sz[0], 1:sz[1]]
    return img

def shave(image, border):
    img = image[border: -border, border: -border]
    return img

In [0]:
#define main prediction function

def predict(image_path):
    
    #load SRCNN model with weights
    srcnn = model()
    srcnn.load_weights('3051crop_weight_200.h5')
    
    #load the degraded and reference images
    path, file = os.path.split(image_path)
    degraded = cv2.imread(image_path)
    ref = cv2.imread('source/{}'.format(file))
    
    #perprocess the image with modcrop
    ref = modcrop(ref, 3)
    degraded = modcrop(degraded, 3)
    
    #convert the image yo YCrCb - (srcnn trained on y channel)
    temp = cv2.cvtColor(degraded, cv2.COLOR_BGR2YCrCb)
    
    #create image slice and normalize
    Y = np.zeros((1, temp.shape[0], temp.shape[1], 1), dtype=float)
    Y[0, :, :, 0] = temp[:, :, 0].astype(float) / 255
    
    #perform super-resolution with srcnn
    pre = srcnn.predict(Y, batch_size = 1)
    
    #post-process output
    pre *= 255
    pre[pre[:] > 255] = 255
    pre[pre[:] < 0] = 0
    pre = pre.astype(np.uint8)
    
    #copy y channel back to image and convert to BGR
    temp = shave(temp, 7)
    temp[:, :, 0] = pre[0, :, :, 0]
    output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2BGR)
    
    #remove border form images
    ref = shave(ref.astype(np.uint8), 7)
    degraded = shave(degraded.astype(np.uint8), 7)

    #image quality calculations
    scores = []
    scores.append(compare_images(degraded, ref))
    scores.append(compare_images(output, ref))
    
    #return images and scores
    return ref, degraded, output, scores


In [0]:
for file in os.listdir('images'):
  ref, degraded, output, scores = predict('images/{}'.format(file))
  
  
  fig, axs = plt.subplots(1, 3, figsize = (20, 8))
  axs[0].imshow(cv2.cvtColor(ref, cv2.COLOR_BGR2RGB))
  axs[0].set_title('Original')
  axs[1].imshow(cv2.cvtColor(degraded, cv2.COLOR_BGR2RGB))
  axs[1].set_title('Degraded')
  axs[1].set(xlabel = 'PSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[0][0], scores[0][1], scores[0][2]))
  axs[2].imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
  axs[2].set_title('SRCNN')
  axs[2].set(xlabel = 'PSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[1][0], scores[1][1], scores[1][2]))
  
  for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])
    
    print(('Saving {}'.format(file)))
    fig.savefig('output/{}.png'.format(os.path.splitext(file)[0]))
    plt.close
  
  
  

NameError: ignored